In [1]:
import os
from IPython.display import HTML

HTML('''<script>
code_show=true;
function code_toggle() {
if (code_show){
$('div.input').hide();
} else {
$('div.input').show();
}
code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
# Description:
#   Exercise7 notebook.
#
# Copyright (C) 2018 Santiago Cortes, Juha Ylioinas
#
# This software is distributed under the GNU General Public 
# Licence (version 2 or later); please refer to the file 
# Licence.txt, included with the software, for details.

# Preparations
import numpy as np

# Select data directory
if os.path.isdir('/coursedata'):
    course_data_dir = '/coursedata'
elif os.path.isdir('../data'):
    course_data_dir = '../data'
else:
    # Specify course_data_dir on your machine
    course_data_dir = '/home/jovyan/work/coursedata/'

print('The data directory is %s' % course_data_dir)
data_dir = os.path.join(course_data_dir, 'exercise-07-data')
print('Data stored in %s' % data_dir)

The data directory is /coursedata
Data stored in /coursedata/exercise-07-data


# CS-E4850 Computer Vision Exercise Round 7
The problems should be solved before the exercise session and solutions returned via
MyCourses. <br><br> For this exercise round, upload this notebook(pdf and .ipynb versions) containing your source codes (for Exercise 1) and your answer to the question of Exercise2, and all the answers to the questions of Exercise 3 (VGG practical), see part[1-3].ipynb. Note that it's not necessary to upload part1.ipynb, part2.ipynb or part3.ipynb, because all of the necessary questions related to them are contained in this notebook and you're not expected to do any coding in Exercises 2 and 3.

## Exercise 1 - Comparing  bags-of-words  with  tf-idf  weighting
Assume  that  we  have  an  indexed  collection  of  documents  containing  the  five  terms  of the following table where the second row indicates the percentage of documents in which each term appears.<br>

| term | cat | dog |mammals | mouse | pet |
| --- | :---: | :---: | :---: | :---: | :---: |
| **% of documents** | 5 | 20 | 2 | 10 | 60 |

Now, given the query $Q=\{mouse, cat, pet, mammals\}$, compute the similarity between $Q$ and the following example documents $D1$, $D2$, $D3$, by using the cosine similarity measure and tf-idf weights (i.e. term frequency - inverse document frequency) for the bag-of-words histogram representations of the documents and the query.

-  $D1$ = Cat is a pet, dog is a pet, and mouse may be a pet too.
-  $D2$ = Cat, dog and mouse are all mammals.
-  $D3$ = Cat and dog get along well, but cat may eat a mouse.

Ignore other words except the five terms. You may proceed with the following steps:

a) Compute and report the inverse document frequency (idf) for each of the five terms. Use the logarithm with base 2. (idf is the logarithm on slide 69 of Lecture 6.)<br>
b) Compute the term frequencies for the query and each document. <br>
c) Form the tf-idf weighted word occurrence histograms for the query and documents. <br>
d) Evaluate the cosine similarity between the query and each document (i.e.\ normalized scalar product between the weighted occurrence histograms as shown on slide 45).<br> 
e) Report the relative ranking of the documents. (You should get similarities 0.95, 0.64, and 0.63, but you need to determine which corresponds to which document.)<br>

In [3]:
## Comparing  bags-of-words  with  tf-idf  weighting
##--your-code-starts-here--##

import math
from pprint import pformat
from collections import Counter

documents = ["Cat is a pet dog is a pet and mouse may be a pet too".lower().split(),
             "Cat dog and mouse are all mammals".lower().split(),
             "Cat and dog get along well, but cat may eat a mouse".lower().split()]

document_histograms = [Counter(d) for d in documents]

terms = ["mouse", "cat", "pet", "mammals"]


In [7]:
# a) Compute and report the inverse document frequency (idf) for each of the five terms. Use the logarithm with base 2.

percentage_in_which_term_appears = {"cat": 1 / 0.05,
                                    "dog": 1 / 0.20,
                                    "mammals": 1 / 0.02,
                                    "mouse": 1 / 0.10,
                                    "pet": 1 / 0.6}

inverse_document_frequency = { t: math.log(percentage_in_which_term_appears[t], 2) for t in terms }

inverse_document_frequency

{'mouse': 3.3219280948873626,
 'cat': 4.321928094887363,
 'pet': 0.7369655941662062,
 'mammals': 5.643856189774724}

In [6]:
# b) Compute the term frequencies for the query and each document.

term_frequencies = {t: [h[t] / sum(h.values()) for h in document_histograms] for t in terms}

term_frequencies

{'mouse': [0.06666666666666667, 0.14285714285714285, 0.08333333333333333],
 'cat': [0.06666666666666667, 0.14285714285714285, 0.16666666666666666],
 'pet': [0.2, 0.0, 0.0],
 'mammals': [0.0, 0.14285714285714285, 0.0]}

In [9]:
# c) Form the tf-idf weighted word occurrence histograms for the query and documents.

tf_idfs = {t: [term_frequencies[t][i] * inverse_document_frequency[t] for i in range(len(documents))] for t in terms}

tf_idfs

{'mouse': [0.22146187299249084, 0.47456115641248037, 0.27682734124061353],
 'cat': [0.2881285396591575, 0.6174182992696232, 0.7203213491478937],
 'pet': [0.14739311883324124, 0.0, 0.0],
 'mammals': [0.0, 0.8062651699678177, 0.0]}

In [16]:
# d) Evaluate the cosine similarity between the query and each document.

def bag2vec(bag, dictionary):
    dict_len = len(list(dictionary.values()))
    vector = [0 for _ in range(dict_len)]
    for term in dictionary.keys():
        vector[dictionary[term]] = bag[term]    
    return vector

def L2_norm(v):
    return math.sqrt(sum([x ** 2 for x in v]))

def dot(a, b):
    assert len(a) == len(b)
    return sum([a[i] * b[i] for i in range(len(a))])

def cosine_similarity(a, b):
    return dot(a, b) / (L2_norm(a) * L2_norm(b))

mappings = {t: i for i, t in enumerate(sorted(terms))}

print('Mappings: {}'.format(pformat(mappings)))


query_vector = bag2vec(Counter(["mouse", "cat", "pet", "mammals"]), mappings)
document_vectors = [bag2vec({t: tf_idfs[t][i] for t in tf_idfs.keys()}, mappings) for i in range(len(documents))]

cosine_similarities = {i: cosine_similarity(dj, query_vector) for i, dj in enumerate(document_vectors)}

print('Cosine similarities: {}'.format(pformat(cosine_similarities)))

Mappings: {'cat': 0, 'mammals': 1, 'mouse': 2, 'pet': 3}
Cosine similarities: {0: 0.8376508896884339, 1: 0.846729872790561, 2: 0.6460861361314962}


In [18]:
# e) Report the relative ranking of the documents.

ranked_cosine_similarities = list(reversed(sorted(cosine_similarities.items(), key=lambda x: x[1])))
rankings = [" ".join(documents[i]) for i, _ in ranked_cosine_similarities]

print('Ranked Cosine Similarities: {}'.format(pformat(ranked_cosine_similarities)))
print('Rankings: {}'.format(pformat(rankings)))

##--your-code-ends-here--##

Ranked Cosine Similarities: [(1, 0.846729872790561), (0, 0.8376508896884339), (2, 0.6460861361314962)]
Rankings: ['cat dog and mouse are all mammals',
 'cat is a pet dog is a pet and mouse may be a pet too',
 'cat and dog get along well, but cat may eat a mouse']


## Exercise 2 - Precision  and  recall
There is a database of 10000 images and a user, who is only interested in images which contain a car. It is known that there are 500 such images in the database. An  automatic image retrieval system retrieves 300 car images and 50 other images from the database. Determine and report the precision and recall of the retrieval  system in this particularcase.

Type your answer here:

Precision and recall are determined as follows:

$$
  \text{precision} = \frac{\text{relevant images}}{\text{returned images}}\\
  \space\\
  \text{recall} = \frac{\text{relevant images}}{\text{total relevant images}}
$$

According to the given information, we can report the following:

$$
  \text{precision} = \frac{300}{350}\\
  \space\\
  \text{recall} = \frac{300}{500}
$$

## Exercise 3 - VGG practical on object instance recognition
See the questions in part[1-3].ipynb and write your answers here.

Part1:
Stage I.A (two questions)
Stage I.B (two questions)
Stage I.C (one question)

Part2 (one question)

Part3:
Stage III.A (three questions)
Stage III.B (one question)
Stage III.C (two questions)

Type your answers here:

# Part I: Sparse features for matching object instances
## Stage I.A: SIFT features detector

<b>Question</b>: Note the change in density of detections across the image. Why does it change? Will it be a problem for matching? How could it be avoided?

Change in density of detections was probably caused by the variance in pixel intensity. For example, in the right image we have a large region with shadows.

This variance probably generates problems for matching: the left image will have more detected features than the right image.

This problem might be solved by lowering the peakThreshold value, which allows lower-contrast features to be detected. However this might result in more noise.


<b>Question</b>: Occasionally, a feature is detected multiple times, with different orientations. This may happen when the orientation assignment is ambiguous. Which kind of image structure would result in ambiguous orientation assignment?

An image structure with an even intensity radial gradient arc, which could have equal intensity along several orientation bins, causing rotational ambiguity (ambiguous orientation assignment).


## Stage I.B: SIFT features detectors and matching between images

<b>Question</b>: Note the descriptors are computed over a much larger region (shown in blue) than the detection (shown in green). Why?

This happens probably because the keypoints only start to become unique at a sufficient scale and descriptor area.


<b>Question</b>: Notice that there are many mismatches. Examine some of the mismatches to understand why the mistakes are being made. For example, is the change in lighting a problem? What additional constraints can be applied to remove the mismatches?

The mismatches might be caused because there are a lot of small features in both images for which there is a large amount of ambiguity in relation to the features in the corresponding image that are actually the correct match.

This problem could be reduced by searching for the nearest neighbour to the matched point and comparing similarity: rejecting the matched feature as a match candidate if the nearest neighbour distance is below some threshold.


## Stage I.C: Improving SIFT matching using Lowe’s second nearest neighbour test

<b>Question</b>: Examine some of the remaining mismatches to understand why they have occurred. How could they be removed?

The remaining mismatches could be removed with some sort of geometric transform fitting technique or RANSAC: we look at all the gradients of the matched feature lines and keep all the inliers, rejecting the outlier gradients which are clearly mismatches.



## Part II: Affine co-variant detectors

<b>Question</b>: The transformation between the images induced by the plane is a planar homography. The detections are only affine co-variant (not as general as a planar homography). So how can descriptors computed on these detections possibly match?

SIFT matching works on local image regions, which are small enough such that the projective transformation can be approximated by an affine transformation. Then, since SIFT is affine-invariant, we can exploit this fact continuing to do the  matching as long as we can approximate the projective transformation with an affine one.



## Part III: Towards large scale retrieval
### Stage III.A: Accelerating descriptor matching with visual words

<b>Questions</b>:
- The size of the vocabulary (the number of clusters) is an important parameter in visual word algorithms. How does the size affect the number of inliers and the difficulty of computing the transformation?

As we increase the size of the vocabulary, we incrase the number of potential matches, as there are more evaluated points that we could potentially match with.

This means that we need to run RANSAC for a longer number of iterations - more specifically we need to run it for $N = \frac{\log(1 - p)}{\log(1 - (1 - e)^s)}$ iterations in order to find enough inliers for a given transformation.

As we detect more and more points, the probability that the points are inliers goes down logarithmically. This means that we need to run RANSAC for a larger number of iterations thus the difficulty of computing the transformation increases.


- In the above procedure the time required to convert the descriptors into visual words was not accounted for. Why?

This time is not accounted because the descriptors are saved into a database and then they are matched either to feature descriptors in the image directly or to the nearest neighbour to those features, which can be looked up in $O(\log(d))$ time on the existing hierarchical tree.


- What is the speedup in searching a large, fixed database of 10, 100, 1000 images?

Taking in account that we are using a tree structure, we can search the inverted index mapping features to images in the tree. We can find the nearest neighbour in logarithmic time and then map to the the relevant database images.

The speedup depends on the number of words in the tree: fewer words means that more database images are going to end up in a bin for a given word and this means that more images need to be searched and aligned. We would expect a 2x, 3x and 4x improvement in performance respectively as the complexity of the k-d tree increases logarithmically with the number of leaf nodes (as opposed to linearly).


### Step III.b: Searching with an inverted index

Task: How many erroneously matched images do you count in the top results?

Around 15.


<b>Question</b>: Why does the top image have a score of 1 (0.9698... see the NOTE above)?

The top image has this score because it matches the query image exactly: it contains all of the same highly-relevant features of the query image.


### Stage III.C: Geometric rescoring

<b>Question</b>: Why is the top score much larger than 1 now?<br>

This top score is now given by the number of inliers to the transformation as opposed to the term-frequency inverse-document-frequency measure: the number of inliers is not a metric but a count. If we could detect more features in an image and it had more inliers, then it is more likely to be a highly relevant or matching image.

<b>Question</b>: Are the retrieval results improved after geometric verification?

Not completely: we still have 14 incorrect matches which were similar in nature to the query but were not an image of the same thing. However, all of the top-ranked matches are now of the same landmark and have substantially different scores to those which were matched purely on features. This shows that the results have improved.